# chapter 3. 전처리
1. 인코딩: 문자열은 모델이 이해를 하지 못한다.
2. 스케일링: 값의 크기가 다르면 공정한 평가를 하지 못한다.
3. 정규화: 각 데이터의 길이를 똑같이 맞추는 방법 (방향성이 중요할때)
4. 데이터 누수: test 데이터로 학습할 경우 발생하는 문제

| 개념 | 핵심 | 코드 |
|------|------|------|
| 인코딩 | 글자→숫자, 순서 없으면 원-핫 | `OneHotEncoder()` |
| 스케일링 | 값 크기 차이 맞추기 | `StandardScaler()`, `MinMaxScaler()` |
| 정규화 | 샘플 길이=1, 방향만 중요할 때 | `Normalizer()` |
| 데이터 누수 | 시험 정보가 훈련에 들어가면 안 됨 | `Pipeline()` |

오늘의 주제:\
a와 b 둘다 같은 데이터와 모델, 하이퍼파라미터를 사용했으나 정확도의 결과가 다르게 나왔다. 왜 그럴까?
-> 이것은 전처리의 차이 때문에 발생한 것이다.

## 0.전처리의 필요성
- 문자열 데이터 = 모델이 계산을 하지 못함 -> 인코딩을 해야한다.
- 값의 크기 차이(단위/범위) = 특정 피처가 지배함 -> 스케일링을 해야한다.
- 길이(크기)보다 방향성이 중요 = 의미/패턴 비교가 어려움 -> 정규화를 해야한다.
- 전처리를 전체 데이터로 fit = 평가가 무효(누수) -> Pipeline를 사용하자.


| 문제 | 결과 | 해결책 |
|-----|-------|--------|
| 문자열 데이터 | 모델이 계산을 하지 못함 | 인코딩 |
| 값의 크기 차이(단위/범위) | 특정 피처가 지배함 | 스케일링 |
| 길이(크기)보다 방향성이 중요 | 의미/패턴 비교가 어려움 | 정규화 |
| 전처리를 전체 데이터로 fit | 평가가 무효(누수) | Pipeline |

In [1]:
import pandas as pd

data = pd.DataFrame({"color": ["red","green","blue","red","green","blue"]})
data

,color
0,red
1,green
2,blue
3,red
4,green
5,blue


## Part 1. 인코딩: 범주형 데이터를 수치형 데이터로 번역
모델은 글자(문자열)을 계산할수 없다.\
예: 색갈 = "빨강", "초록", "파랑"

문자열을 그대로 넣으면 모델은 해당 요소를 파악하지 못한다.\
때문에 문자를 숫자로 **번역**하는 과정을 거쳐야 한다.\
이 과정을 **인코딩**이라 한다.

### 명목형 인코딩 (순서가 없는): 레이블 vs 원-핫
1) 레이블 인코딩 (Label Encoding)
    - red = 0, green = 1, blue = 2 처럼 각 요소에 번호를 부착한다.
    - 문제: 0, 1, 2와 같이 **숫자 크기**에 의미를 붙이는 모델들이 많기 때문에 사용에 주의해야 한다.
2) 원-핫 인코딩 (One-Hot Encoding)
    - red=[1,0,0]처럼 존재 유무로 표현한다.
    - red = [1,0,0]
    - green = [0,1,0]
    - blue = [0,0,1]
    - 순서를 만들지 않고, 단지 존재 유무만 표현한다.
    - 레이블 인코딩의 단점을 보안(숫자 크기의 유무에 의한 왜곡, 착시를 방지)

> 레이블은 숫자 크기에 의미가 생긴 것처럼 보일 수 있다.(가짜 순서).
> 원-핫은 순서를 만들지 않는다.

#### 차원(Dimension)이란?
차원이란 모델에 들어가는 컬럼(피처)의 개수라 보면 된다.\
원-핫의 경우 red = [1,0,0], green = [0,1,0], blue = [0,0,1] 로 구성되어 차원이 늘어난다.

color 1개 컬럼이면 1차원\
원-핫으로 red/green/blue 컬럼을 만들면 3차원

차원이 늘어나게 되면 데이터 구조는 더 복잡하게 되고 컬럼이 폭발한다.(메모리/학습 시간 증가)\
이로 인해 모델이 복잡해져 과적합의 위험성이 늘어나기에, 차원을 줄이거나 중요한 축만 남기는 방법도 중요하다.

### 코드 사용 예시

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 레이블 인코딩
le = LabelEncoder()
data["color_label"] = le.fit_transform(data["color"])

# 원-핫 인코딩
ohe = OneHotEncoder(sparse_output=False)
ohe_matrix = ohe.fit_transform(data[["color"]])
ohe_df = pd.DataFrame(ohe_matrix, columns=ohe.categories_[0])

data_ohe = pd.concat([data, ohe_df], axis=1)

print("="*50)
print("레이블 인코딩")
display(data)

print("="*50)
print("원-핫 인코딩")
display(data_ohe)
print("="*50)

레이블 인코딩


,color,color_label
0,red,2
1,green,1
2,blue,0
3,red,2
4,green,1
5,blue,0


원-핫 인코딩


,color,color_label,blue,green,red
0,red,2,0.0,0.0,1.0
1,green,1,0.0,1.0,0.0
2,blue,0,1.0,0.0,0.0
3,red,2,0.0,0.0,1.0
4,green,1,0.0,1.0,0.0
5,blue,0,1.0,0.0,0.0


### 순서형(작<중<대): 오디널 인코딩
**순서가 의미 있을 때**만 오디널 인코딩을 쓴다.\
오디널은 "순서"를 사람이 지정해주는것이 핵심이다.

In [8]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

df_ord = pd.DataFrame({"size":["small","medium","large","medium","small"]})
ord_enc = OrdinalEncoder(categories=[["small","medium","large"]])
df_ord["size_code"] = ord_enc.fit_transform(df_ord[["size"]])

print("="*50)
print("오디널 인코딩")
display(df_ord)
print("="*50)


오디널 인코딩


,size,size_code
0,small,0.0
1,medium,1.0
2,large,2.0
3,medium,1.0
4,small,0.0


### 인코딩 선택 규칙 (절대적이지 않음)

데이터가 범주형 데이터인가?
- 순서가 있는가? (small < medium < large)
    - yes → Ordinal (순서가 의미 있을때)
    - no
        - 어떤 모델을 사용하는가?
            - 트리 모델 → (One-Hot 권장 / 순서형이면 Ordinal / Label 도 가능)
            - 선형/거리 기반: OneHot이 가장 안정


| 데이터 유형           | 추천 인코딩  | 장점                 | 주의점                 |
| ---------------- | ------- | ------------------ | ------------------- |
| 명목형(순서 없음)       | 원-핫     | 가짜 순서 없음, 기본값으로 안전 | 카테고리 많으면 차원 폭발      |
| 순서형(순서 있음)       | 오디널     | 순서 반영 가능           | 순서를 직접 지정해야 함       |
| (예외) 트리 모델에서 명목형 | 레이블도 가능 | 구현이 간단             | 선형/거리 모델에는 가짜 순서 위험 |

### 텍스트 인코딩 (간략하게)
짧은 범주형 데이터가 아닌 문장(텍스트)는 어떻게 접근해야 할까?

문장은 보통 단어 기준으로 백터로 바꾼다.\
CountVectorizer는 단어 빈도를 세서 벡터로 만든다.

> 단어 기준으로 백터화 시킨다.

## Part 2. 스케일링 (Scaling)
피처마다 값 범위가 너무 다르면, 큰 범위 피처가 모델을 지배할 수 있다.\
특히 거리 기반/경사 기반 모델에서 문제가 자주 생긴다.

### 예시 코드
학생 키 = [150cm, 160cm, 170cm] → 범위: 20\
학생 용돈 = [1000원, 20000원, 50000원] → 범위: 49000\

용돈 값이 너무 커서 모델은 키를 무시하게 된다.\
이로 인해 거리 계산에서 용돈이 거의 모든 것을 결정해버릴 수 있다.

In [10]:
import numpy as np

# [키(cm), 용돈(원)]
A = np.array([150, 1000])
B = np.array([170, 20000])

dist = np.sqrt(((A - B) ** 2).sum())
print(dist)

19000.010526312875


위 거리에서 (170-150)=20은 제곱해도 400인데,\
(20000-1000)=19000은 제곱하면 361,000,000 으로 결과적으로 용돈이 거의 전부를 결정한다.

### 대표적인 스케일러 1: StandardScaler
평균 0, 표준편차 1로 맞춘다(z-score).\
이상치에 민감할 수 있다(평균/표준편차가 흔들림).

```
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
```

In [11]:
import numpy as np
from sklearn.preprocessing import StandardScaler

X = np.array([
    [150,1000],
    [160,20000],
    [170,50000]
])

# 평균 0, 표준편차 1 (-1 ~ 1)
# 모델이 공평하게 학습(비교도 하구)
X_scaled = StandardScaler().fit_transform(X)

df_scale = pd.DataFrame(
    np.hstack([X, X_scaled]),
    columns=["키(cm)","용돈(원)","키(스케일링 후)","용돈(스케일링 후)"]
)
df_scale

,키(cm),용돈(원),키(스케일링 후),용돈(스케일링 후)
0,150.0,1000.0,-1.224745,-1.123698
1,160.0,20000.0,0.000000,-0.181775
2,170.0,50000.0,1.224745,1.305473


스케일링 이후 "키"와 "용돈"이 비슷한 범위로 맞춰진다.\
이로 인해 두 특정을 공정하게 반영할 수 있다.

### 대표적인 스케일러 2: RobustScaler
중앙값(median)과 IQR(Q3-Q1)을 사용한다.\
이상치가 있어도 평균/표준편차처럼 크게 흔들리지 않는다. (이상치에 강하다.)


```
RobustScaler().fit_transform(X)
```

In [12]:
import numpy as np, pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler

X = np.array([[1., 10.],
              [2., 12.],
              [3., 13.],
              [100., 1000.]])   # 이상치
# 평균 = 0, 표준편차 1로 변환 (z-Score) / 평균을 사용하기에 이상치에 민감
sc = StandardScaler().fit_transform(X)
# 중앙값, IQR을 가지고 스케일링 / 이상치에 상대적으로 덜 민감
rb = RobustScaler().fit_transform(X)

df_rb = pd.DataFrame(
    np.hstack([X, sc, rb]),
    columns=["x1", "x2", "Std_x1", "Std_x2", "Robust_x1", "Robust_x2"]
)
df_rb

,x1,x2,Std_x1,Std_x2,Robust_x1,Robust_x2
0,1.0,10.0,-0.600832,-0.581243,-0.058824,-0.010070
1,2.0,12.0,-0.577270,-0.576570,-0.019608,-0.002014
2,3.0,13.0,-0.553708,-0.574233,0.019608,0.002014
3,100.0,1000.0,1.731810,1.732045,3.823529,3.977845


robust 결과는 각 컬럼에서 중앙값을 0 근처로 만들고, IQR 기준으로 스케일을 맞춘 값이다.\
이상치가 있어도 중앙값/IQR이 상대적으로 덜 흔들려서 일반 구간 값들이 덜 찌그러진다.

### 대표적인 스케일러 3: MinMaxScaler
최소값을 0, 최대값을 1로 맞춘다.\
범위를 딱 [0, 1]로 고정하고 싶을 때 직관적이다.\
만약 이상치가 있으면 대부분 값이 한쪽으로 몰릴 수 있다.

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

X = np.array([
    [150,  1000],
    [160, 20000],
    [170, 50000]
])

X_mm = MinMaxScaler().fit_transform(X)

df_mm = pd.DataFrame(
    np.hstack([X, X_mm]),
    columns=["키(cm)", "용돈(원)", "키(minmax)", "용돈(minmax)"]
)
df_mm

,키(cm),용돈(원),키(minmax),용돈(minmax)
0,150.0,1000.0,0.0,0.000000
1,160.0,20000.0,0.5,0.387755
2,170.0,50000.0,1.0,1.000000


키(minmax)는 150~170 사이를 0~1로 바꾼 값이다.\
용돈(minmax)는 1000~50000 사이를 0~1로 바꾼 값이다.\

두 피처가 같은 0~1 범위가 되어서 거리/학습에서 공정해진다.

### 스케일링 선택 규칙 (절대적이지 않음)

- 데이터에 이상치가 있는가?
    - YES → RobustScaler
    - NO
        - 값의 범위가 중요한가?
            - YES → MinMaxScaler
            - NO  → StandardScaler (가장 일반적)


| 방법             | 무엇으로 맞추나      | 좋을 때                | 문제/주의          |
| -------------- | ------------ | ------------------- | -------------- |
| StandardScaler | 평균 0, 표준편차 1 (z-score) | 보통 기본값, 값 분포가 무난할 때 | 이상치에 민감        |
| RobustScaler   | 중앙값, IQR     | 이상치가 섞일 때 (이상치에 강함)          | 분포가 특이하면 해석 주의 |
| MinMaxScaler   | [0, 1] 구간으로 변환       | 범위를 고정하고 싶을 때       | 이상치가 있으면 몰림 현상 |

## Part 3. 정규화(normalization)
각 샘플(행) 벡터의 길이를 1로 맞춰서, 총량(길이) 영향은 지우고 패턴(방향)만 비교하게 만드는 전처리\
정규화는 길이를 1로 만들어 **길이 영향은 지우고, 방향(패턴)만 비교** 하는 것이다.

- 백터의 해석
    - **길이(크기)**: 값이 전체적으로 얼마나 큰가 (예: 문장 길이, 단어 총량)
    - **방향(패턴)**: 값들의 비율/조합이 어떤가 (예: 어떤 단어들이 상대적으로 더 많이 나오나)

예시 1) 숫자 벡터\
벡터: [3, 4]\
길이(L2 norm): 5\
정규화: [3/5, 4/5] = [0.6, 0.8]

예시 2) 취향(비율) 비유\
A: 록 10곡, 재즈 5곡 → [10, 5]\
B: 록 100곡, 재즈 50곡 → [100, 50]
→ 총량은 다르지만 비율은 동일하다.(2:1)\
→ 정규화하면 같은 패턴으로 더 가깝게 본다.

### 코드 해설
```
normalizer = Normalizer()       # 정규화 도구 생성
X_norm = normalizer.fit_transform(X)
```


In [15]:
from sklearn.preprocessing import Normalizer

X = np.array([[3,4],
              [1,2],
              [10,0]])

normalizer = Normalizer()
X_norm = normalizer.fit_transform(X)

df_norm = pd.DataFrame(
    np.hstack([X, X_norm]),
    columns=["x1","x2","x1(정규화 후)","x2(정규화 후)"]
)
df_norm

,x1,x2,x1(정규화 후),x2(정규화 후)
0,3.0,4.0,0.600000,0.800000
1,1.0,2.0,0.447214,0.894427
2,10.0,0.0,1.000000,0.000000


### 정규화와 코사인 유사도(핵심만)
코사인 유사도(cosine similarity)는 두 벡터의 방향이 얼마나 비슷한지 본다.
같은 방향이면 1, 반대 반향이면 -1, 직각에 가까우면 0(관련 거의 없음)

벡터를 L2 정규화(길이 1)하면, 코사인 유사도는 내적(dot)과 같아진다.\
즉, 정규화는 "코사인 유사도 같은 방향 비교"를 깔끔하게 만들어 준다. (정규화 + 내적(=코사인)) 조합이 자주 등장

In [ ]:
import numpy as np
from sklearn.preprocessing import Normalizer
from numpy.linalg import norm

A = np.array([[3.,4.]])
B = np.array([[6.,8.]])

normer = Normalizer()
A_n = normer.fit_transform(A); B_n = normer.transform(B)

cos = (A @ B.T) / (norm(A) * norm(B))
dot_after_norm = (A_n @ B_n.T)
print(f"cosine(A,B)= {cos.item()}")
print(f"dot(normalized)= {dot_after_norm.item()}")

### 문장 유사도 — CountVectorizer + 정규화 + 코사인 유사도
문장을 벡터로 바꾼 뒤(CountVectorizer), 길이를 맞추고(Normalizer), 방향이 얼마나 비슷한지(코사인 유사도)로 문장 유사도를 계산한다.

1. CountVectorizer: 문장 → 단어 개수 벡터
2. Normalizer: 벡터 길이 1로(총량 제거, 패턴만)
3. cosine_similarity: 방향이 비슷하면 1에 가까움

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity

texts = [
    "영화가 좋다",
    "영화가 진짜로 좋다"
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)  # 1) 문장 -> 단어 빈도 벡터

print("단어 사전:", vectorizer.get_feature_names_out())

normalizer = Normalizer()
X_norm = normalizer.fit_transform(X) # 2) 행(문장) 길이 정규화

sim = cosine_similarity(X_norm[0], X_norm[1]) # 3) 코사인 유사도
print("코사인 유사도:", sim[0][0])

단어 사전: ['영화가' '좋다' '진짜로']
코사인 유사도: 0.8164965809277261


cosine_similarity(a, b): 두 벡터 방향 유사도 계산

코사인 유사도가...\
1에 가까울수록 두 문장이 비슷함\
0에 가까울수록 거의 안 비슷함

### 스케일링과 정규화 차이
스케일링: 피처(열) 단위로 값 범위를 맞춘다.\
정규화: 샘플(행) 단위로 벡터 길이를 1로 만든다(방향만 본다).

포인트?
- 기준이 다르다
    - 스케일링 = 열 기준
    - 정규화 = 행 기준
- 목적이 다르다
    - 스케일링 = 변수 크기 차이 보정(공평한 학습)
    - 정규화 = 총량 제거, 패턴/유사도(방향) 비교

| 구분   | 단위    | 목적           | 언제 쓰나           |
| ---- | ----- | ------------ | --------------- |
| 스케일링 | 피처(컬럼) | 변수 크기 차이 보정  | 거리/경사에 민감한 모델   |
| 정규화  | 샘플(행) | 총량 제거, 방향 비교<br> 벡터 길이를 1로   | 텍스트/임베딩/코사인 유사도 |

## Part 4. 데이터 누수와 파이프라인

### 데이터 누수란?\
훈련 과정에서 보면 안 되는 test 정보를 미리 써버리는 실수다.\
전처리 단계에서 특히 자주 생긴다.(스케일러 fit을 전체 데이터에 해버리는 경우)


특히 스케일링 같은 전처리는 fit 과정에서 통계량(평균/표준편차, min/max 등)을 계산한다.\
그런데 전체 데이터(X 전체)로 fit하면 test 데이터의 통계량까지 섞여 들어간다.\
즉, 모델이 시험지를 살짝 본 상태로 평가하는 꼴이 된다.

1. 성능이 과장돼 보인다(실험 결과 신뢰 불가)
2. 모델 선택이 틀릴 수 있다
3. 배포 후 성능 하락으로 이어진다

이로 인해 실제 신규 데이터에서는 성능이 유지되지 않는다.
> 때문에 전처리는 반드시 훈련셋으로만 fit 해야한다.

In [19]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

scaler_bad = StandardScaler().fit(X)      # 전체 X로 fit → 누수
X_train_bad = scaler_bad.transform(X_train)
X_test_bad  = scaler_bad.transform(X_test)

model = KNeighborsClassifier(n_neighbors=5).fit(X_train_bad, y_train)
pred = model.predict(X_test_bad)

print("Acc:", accuracy_score(y_test, pred))

Acc: 0.9111111111111111


fit(X)가 문제다. 여기서 test 정보가 섞여 데이터 누수가 발생한다.

정확도가 0.911... 임에도 데이터누수가 발생했기에 해당 정확도는 신뢰하기 어렵다.

### 데이터누수 해결 방법: Pipeline으로 전처리+모델을 묶기
Pipeline이란?\
전처리와 모델을 한 덩어리로 묶어서, fit/predict 흐름을 실수 없이 유지하게 만든다.


```python
pipe = Pipeline([
    ("scaler", StandardScaler()),  # 전처리
    ("model", LogisticRegression()) # 모델
])

pipe.fit(X_train, y_train)   # 훈련셋으로 fit
pipe.predict(X_test)         # 시험셋은 transform만
```
Pipeline([...]): 전처리+모델 묶기



Pipeline을 쓰면 "전처리는 train에만 fit" 규칙을 자연스럽게 지키게 된다.

In [20]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

pipe.fit(X_train, y_train)     # scaler는 train으로만 fit
pred = pipe.predict(X_test)    # test는 transform만

### 교차검증에서 누수가 더 잘 터지는 이유
교차검증은 fold마다 train/test가 계속 바뀐다.\
근데 미리 전체 X를 스케일링해버리면, 각 fold의 test(시험 데이터) 정보가 스케일러에 들어가게 된다\
→ 데이터 누수 발생.

In [21]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 누수: 전체 X로 스케일러 fit 후 CV
X_scaled_bad = StandardScaler().fit_transform(X)
scores_bad = cross_val_score(KNeighborsClassifier(n_neighbors=5), X_scaled_bad, y, cv=cv)

# 정상: Pipeline이면 fold마다 train에만 fit
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])
scores_pipe = cross_val_score(pipe, X, y, cv=cv)

print("누수 mean:", scores_bad.mean())
print("pipe mean:", scores_pipe.mean())

누수 mean: 0.9666666666666666
pipe mean: 0.9733333333333334


결과 해석
- 누수된 전처리 방식: 시험 데이터 정보를 슬쩍 본 상태기에 성능이 더 높게 나올 수 있음
- Pipeline 방식: 훈련 데이터에만 fit → 시험 데이터에만 transform → 더 공정하고 현실적인 성능


실제 데이터에서는 누수 방식이 **성능을 뻥튀기**해서 보여주는 경우가 많으나\
그것이 예측/분류 성능이 높다는것을 의미하지 않는다.

> - **Pipeline**을 습관처럼 사용해야 안전하다